In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


def Belief(State,p,r):
    
    equilibrium = r/(1-p+r);
    
    if State[0]== 0 and State[1]> 0:
            Belief = (r-r*(p-r)**(State[1]))/(1+r-p);
    elif State[0]== 1 and State[1]> 0:
            Belief = (r+(1-p)*(p-r)**(State[1]))/(1+r-p);
            
    else:
        
        Belief = equilibrium;

    return Belief;


def TransitionMatrix(t,rho,p,r):
    M = np.zeros((t+2,t+2));
    for i in range (t-1):
        M[i,i+1]=1;
    M[t-1,0]= rho*(1-Belief([0,t],p,r));
    M[t-1,t]= 1-rho;
    M[t-1,t+1]= rho*Belief([0,t],p,r);
    M[t,0]= 1-Belief([0,t+1],p,r);
    M[t,t+1]= Belief([0,t+1],p,r);
    M[t+1,0]= 1-Belief([1,1],p,r);
    M[t+1,t+1]= Belief([1,1],p,r);
    return M;




In [3]:
print(TransitionMatrix(4,0.4,0.7,0.4));

[[0.      1.      0.      0.      0.      0.     ]
 [0.      0.      1.      0.      0.      0.     ]
 [0.      0.      0.      1.      0.      0.     ]
 [0.17328 0.      0.      0.      0.6     0.22672]
 [0.42996 0.      0.      0.      0.      0.57004]
 [0.3     0.      0.      0.      0.      0.7    ]]


In [2]:
def steady_state_prop(p):
    dim = p.shape[0]
    q = (p-np.eye(dim))
    ones = np.ones(dim)
    q = np.c_[q,ones]
    QTQ = np.dot(q, q.T)
    bQT = np.ones(dim)
    return np.linalg.solve(QTQ,bQT)



In [3]:
def find_threshold(alpha,p,r):
    threshold=0;
    while threshold >=0:
        steadystate= steady_state_prop(TransitionMatrix(threshold,0.0001,p,r));
        active= steadystate[threshold-1]*0.0001+steadystate[threshold]+steadystate[threshold+1];
        if active< alpha:
            break
        threshold += 1;
    return threshold;


t=find_threshold(0.4,0.7,0.4);
print(t);




5


In [7]:
steadystate= steady_state_prop(TransitionMatrix(5,0.4,0.7,0.4));
print(steadystate);
steadystate= steady_state_prop(TransitionMatrix(4,0.4,0.7,0.4));
print(steadystate);

[0.13329641 0.13329641 0.13329641 0.13329641 0.13329641 0.07997785
 0.25354009]
[0.15394531 0.15394531 0.15394531 0.15394531 0.09236719 0.29185157]


In [4]:
def find_rho(alpha,p,r):
    t= find_threshold(alpha,p,r);
    h=1;
    ind=1;
    
    for i in range(10000):
        M = TransitionMatrix(t,i*0.0001,p,r);
        steadystate= steady_state_prop(M);
        active= steadystate[t-1]*i*0.0001+steadystate[t]+steadystate[t+1];
        if abs(active-alpha)<h:
            h= abs(active-alpha);
            ind= i;
            
    return ind*0.0001;
        
    

In [6]:
print(find_rho(0.4,0.85,0.25));

0.3267


In [5]:
def upperbound(alpha,p,r):
    t = find_threshold(alpha,p,r);
    rho = find_rho(alpha,p,r);
    steadystate= steady_state_prop(TransitionMatrix(t,rho,p,r));
    upperbound= steadystate[t-1]*Belief([0,t],p,r)*rho+steadystate[t]*Belief([0,t+1],p,r)+steadystate[t+1]*p;
    return upperbound*100;





In [6]:
print(upperbound(0.4,0.6,0.3));

20.61379783082291


In [7]:
print(upperbound(0.4,0.7,0.4));

26.212987902872914


In [6]:
print(upperbound(0.5,0.4,0.3));

17.799150170468227


In [7]:
print(upperbound(0.5,0.6,0.2));

21.604805762553628


In [8]:
print(upperbound(0.7,0.9,1/15));

38.00000000000003


In [12]:
print(upperbound(0.7,0.5,1/3));

30.0


In [17]:
print(upperbound(0.2,0.65,0.15));

9.215367310792812


In [18]:
print(upperbound(0.2,0.44,0.24));

6.976722524031495


In [19]:
print(upperbound(0.2,0.9,1/15));

15.92382643743303


In [20]:
print(upperbound(0.2,0.5,1/3));

8.888884870725507


In [10]:
print(upperbound(0.4,0.7,0.1));

16.360464578247345


In [11]:
print(upperbound(0.4,0.8,0.2));

28.190534168558933


In [12]:
print(upperbound(0.4,0.85,0.25));

32.18066479703865


In [14]:
print(upperbound(0.4,0.8,0.55));

31.427973225263884


In [15]:
print(upperbound(0.4,0.9,0.6));

35.8209594319144


In [16]:
print(upperbound(0.4,0.4,0.2));

11.704486806085837
